In [1]:
import keras
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from pennylane import AngleEmbedding, StronglyEntanglingLayers, RandomLayers
import pennylane as qml
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tensorflow.keras.layers import Layer
import tensorflow as tf
import time

2023-12-18 18:30:01.139935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 18:30:01.140003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 18:30:01.140650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 18:30:01.144686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 18:30:01.732977: W tensorflow/compiler/tf2

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


In [3]:
x_train = x_train[:10_000]
y_train = y_train[:10_000]

x_test = x_test[:1_000]
y_test = y_test[:1_000]

In [4]:
y_test = y_test.astype(np.float32)
y_train = y_train.astype(np.float32)

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [5]:
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

2023-12-18 18:30:03.424262: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 18:30:03.446295: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 18:30:03.446355: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 18:30:03.448940: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 18:30:03.448998: I external/local_xla/xla/stream_executor

In [6]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
channel_size = 3
num_qubits = 9 # use 3 x 3 qubits for kernel 

In [8]:
class QuantumConvLayer(Layer):
    def __init__(self, filter, circuit_length, padding_mode: str, **kwargs):
        super(QuantumConvLayer, self).__init__(**kwargs)
        self.filter = filter
        self.circuit_length = circuit_length
        self.padding_mode = padding_mode
        
        

    def build(self, input_shape):
        self.num_qubits = input_shape[-1] * 4
        self.device = qml.device("lightning.qubit", wires=self.num_qubits)
        
        @qml.simplify
        @qml.qnode(self.device, interface="tf")
        def quantum_conv_circuit(inputs, weights):
            AngleEmbedding(inputs, wires=range(self.num_qubits))
            RandomLayers(weights, wires=range(self.num_qubits))

            return qml.expval(qml.PauliZ(wires=0))

        
        quanv_circ = [qml.qnn.KerasLayer(quantum_conv_circuit, {"weights": (self.circuit_length, self.num_qubits)}, output_dim=1) for _ in range(self.filter)]
        
        outputs = []
        input_layer = keras.Input(self.num_qubits)
        for layer in quanv_circ:
            layer.trainable = True
            outputs.append(layer(input_layer))
            
        output = keras.layers.Concatenate(axis=-1)(outputs)
        
        self.filter_stage = keras.Model(inputs=input_layer, outputs=output, name="filter_stage")
        
    @tf.function
    def apply_on_batch(self, inputs):
        
        return self.filter_stage(inputs)
        
    @tf.function
    def call(self, inputs):      
        
        patches = tf.image.extract_patches(
            images=inputs,
            sizes=[1, 2, 2, 1],
            strides=[1, 1, 1, 1],
            rates=[1, 1, 1, 1],
            padding=self.padding_mode
        )
        patches_flat = tf.reshape(patches, [tf.shape(patches)[0], patches.shape[1] * patches.shape[2], patches.shape[3]])
        
        output = tf.map_fn(self.apply_on_batch, patches_flat, dtype=tf.float32)
        #output = self.conv_stage(patches_flat)
        output_shape = tf.shape(inputs)
        
        output = tf.reshape(output, [output_shape[0], output_shape[1], output_shape[2], self.filter])
        return output


In [9]:
model = keras.Sequential(
    [
        keras.layers.Conv2D(3, 3, strides=(2, 2), activation="linear"),
        QuantumConvLayer(6, 2, "SAME"),
        keras.layers.Flatten(),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ]
)

In [10]:
_ = model(np.random.random((1, 28, 28, 1)))

Instructions for updating:
Use fn_output_signature instead
Cause: Unable to locate the source code of <function _gcd_import at 0x7f7bd8fd7400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function _gcd_import at 0x7f7bd8fd7400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source 

2023-12-18 18:30:27.159185: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-18 18:30:27.228036: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-18 18:30:28.274936: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (1, 13, 13, 3)            30        
                                                                 
 quantum_conv_layer (Quantu  (1, 13, 13, 6)            144       
 mConvLayer)                                                     
                                                                 
 flatten (Flatten)           (1, 1014)                 0         
                                                                 
 dense (Dense)               (1, 16)                   16240     
                                                                 
 dense_1 (Dense)             (1, 10)                   170       
                                                                 
Total params: 16584 (64.78 KB)
Trainable params: 16584 (64.78 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [13]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer, loss=loss_fn, metrics=metrics)

In [ ]:
fitting = model.fit(x_train, y_train, epochs=10, batch_size=4, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10


2023-12-18 18:30:47.286434: I external/local_xla/xla/service/service.cc:168] XLA service 0x2e667bb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-18 18:30:47.286469: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
I0000 00:00:1702920647.327727   26604 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 204/2500 [=>............................] - ETA: 7:45:38 - loss: 2.3064 - sparse_categorical_accuracy: 0.1201